PROJETO REALIZADO EM PARCERIA COM TAMIRA ABOGANEM

Começamos o código fazendo a instalação dos componentes necessários e importação dos módulos pertinentes.
Depois lemos o arquivo CSV

In [ ]:
%pip install pandas
%pip install pandasql
%pip install sqlalchemy
%pip install psycopg2

In [2]:
import pandas as pd
import pandasql as ps
from sqlalchemy import create_engine


In [ ]:
df = pd.read_csv("C:/Projeto/imdb_top250_movies.csv")
df

In [ ]:
top_filmes = pd.read_csv("C:/Projeto/imdb_top250_movies.csv")
top_filmes


Agora que já fizemos a leitura do arquivo, vamos dar uma olhada na estrutura e tipo de informações que ele tem.
Já sabemos que há colunas com diferentes tipagens: string, integer, float, data, tempo, moeda etc.
Para ter uma noção do tamanho do arquivo, podemos ver seu shape, isto é, seu formato.
Nesse caso, são 250 linhas e 23 colunas.

In [ ]:
df.shape

Com um simples describe, já podemos ver algumas informações interessantes:
1. O filme mais antigo da nossa listagem foi lançado em 1921 e o mais recente em 2017.
2. A pontuação máxima dos 250 melhores filmes é de 9.3, a mínima de 8.0 e a média de 8.24.



In [ ]:
df.describe()

,Unnamed: 0,Num,Year,Metascore,imdbRating
count,250.000000,250.000000,250.000000,177.000000,250.000000
mean,124.500000,125.500000,1982.676000,80.045198,8.244000
std,72.312977,72.312977,24.809212,11.030340,0.245735
min,0.000000,1.000000,1921.000000,53.000000,8.000000
25%,62.250000,63.250000,1961.250000,72.000000,8.100000
50%,124.500000,125.500000,1988.000000,81.000000,8.200000
75%,186.750000,187.750000,2003.000000,89.000000,8.375000
max,249.000000,250.000000,2017.000000,100.000000,9.300000


In [14]:
from sqlalchemy import create_engine

SGBD = "postgresql+psycopg2"
USER = "postgres"
SENHA = "senha"
HOST = "localhost"
DATABASE = "postgres"

engine = create_engine(f'{SGBD}://{USER}:{SENHA}@{HOST}/{DATABASE}')
conn = engine.connect()

df.to_sql("df", con=engine)


250

1. Fazendo uma busca simples, já podemos identificar quais diretores, produtoras e categorias aparecem no top 10 da lista.
Basta exibir o nome, categoria, diretor e produtora dos 10 primeiros colocados:

In [ ]:
query = """
SELECT 
  Title,
  Genre,
  Director,
  Production,
  Year,
  imdbRating AS "Pontuação"
FROM
  top_filmes
ORDER BY imdbRating DESC
LIMIT 10
"""

ps.sqldf(query)

2. E agora as mesmas informações sobre os 10 últimos colocados:

In [ ]:
query = """
SELECT
  Title,
  Genre,
  Director,
  Production,
  Year,
  imdbRating AS "Pontuação"
FROM
  top_filmes
ORDER BY imdbRating DESC
LIMIT 10
"""

ps.sqldf(query)

3. Para calcular a média de pontuação do top 10, podemos:

In [ ]:
query = """
SELECT
  AVG("Pontuação") AS "Pontuação Média"
FROM (SELECT
        Title,
        Genre,
        Director,
        Production,
        Year,
        imdbRating AS "Pontuação"
      FROM
        top_filmes
      ORDER BY imdbRating DESC
      LIMIT 10) AS TOP10
GROUP BY AVG("Pontuação")
"""

ps.sqldf(query)

4. Contando quantas vezes um mesmo diretor aparece na lista, sabemos quais os mais premiados:

In [ ]:
query = """
SELECT
  Director,
  COUNT(Title)
FROM
  top_filmes
ORDER BY COUNT(Title) DESC
"""

ps.sqldf(query)

5. Descobrindo as categorias de filme que existem na nossa lista:


In [ ]:
query = """
SELECT
  DISTINCT Genre
FROM
  top_filmes
ORDER BY COUNT(Title) DESC
"""

ps.sqldf(query)

7. E o tempo de duração do filme, será que tem a ver com pontuação e votação? Vejamos:

In [ ]:
query = """
SELECT
  Num,
  Title,
  imdbVotes,
  Runtime
FROM
  top_filmes
ORDER BY Runtime
"""

ps.sqldf(query)

8. Média de votos para os filmes da lista:

In [ ]:
query = """
SELECT
  SUM(imdbVotes) AS "Votação Média"
FROM
  top_filmes
GROUP BY SUM(imdbVotes)
"""

ps.sqldf(query)

9. Quais os anos que produziram os melhores filmes?

In [ ]:
query = """
SELECT
  Title,
  Year,
FROM
  top_filmes
ORDER BY imdbRating DESC
LIMIT 10
"""

ps.sqldf(query)

10. Por último, mas não menos importante, podemos identificar os melhores escritores e atores:

In [ ]:
query = """
SELECT
  Title,
  Writer,
  Actors
FROM
  top_filmes
ORDER BY imdbRating DESC
LIMIT 10
"""

ps.sqldf(query)